# BEE 4750 Homework 4: Linear Programming and Capacity Expansion

**Name**: Raashid Husain

**ID**:RAH383

> **Due Date**
>
> Thursday, 10/31/24, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to formulate and solve a resource allocation
    problem using linear programming.
-   Problem 2 asks you to formulate, solve, and analyze a standard
    generating capacity expansion problem.
-   Problem 3 (5750 only) asks you to add a CO<sub>2</sub> constraint to
    the capacity expansion problem and identify changes in the resulting
    solution.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [5]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Downloads/BEE 4750/hw4-raashid-husain`


In [6]:
using JuMP
using HiGHS
using DataFrames
using Plots
using Measures
using CSV
using MarkdownTables

## Problems (Total: 50/60 Points)

### Problem 1 (20 points)

A farmer has access to a pesticide which can be used on corn, soybeans,
and wheat fields, and costs \$70/kg-yr to apply. The crop yields the
farmer can obtain following crop yields by applying varying rates of
pesticides to the field are shown in
<a href="#tbl-yields" class="quarto-xref">Table 1</a>.

| Application Rate (kg/ha) | Soybean (kg/ha) | Wheat (kg/ha) | Corn (kg/ha) |
|:------------------------:|:---------------:|:-------------:|:------------:|
|            0             |      2900       |     3500      |     5900     |
|            1             |      3800       |     4100      |     6700     |
|            2             |      4400       |     4200      |     7900     |

Table 1: Crop yields from applying varying pesticide rates for Problem
1.

The costs of production, *excluding pesticides*, for each crop, and
selling prices, are shown in
<a href="#tbl-costs" class="quarto-xref">Table 2</a>.

|   Crop   | Production Cost (\$/ha-yr) | Selling Price (\$/kg) |
|:--------:|:--------------------------:|:---------------------:|
| Soybeans |            350             |         0.36          |
|  Wheat   |            280             |         0.27          |
|   Corn   |            390             |         0.22          |

Table 2: Costs of crop production, excluding pesticides, and selling
prices for each crop.

Recently, environmental authorities have declared that farms cannot have
an *average* application rate on soybeans, wheat, and corn which exceeds
0.8, 0.7, and 0.6 kg/ha, respectively. The farmer has asked you for
advice on how they should plant crops and apply pesticides to maximize
profits over 130 total ha while remaining in regulatory compliance if
demand for each crop (which is the maximum the market would buy) this
year is 250,000 kg?

**In this problem**:

-   Formulate a linear program for this resource allocation problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations). **Tip: Make sure that all of your constraints are
    linear**.
-   Implement the program in `JuMP.jl` and find the solution. How many
    ha should the farmer dedicate to each crop and with what pesticide
    application rate(s)? How much profit will the farmer expect to make?
-   The farmer has an opportunity to buy an extra 10 ha of land. How
    much extra profit would this land be worth to the farmer? Discuss
    why this value makes sense and whether you would recommend the
    farmer should make the purchase.

In [37]:
crop_model = Model(HiGHS.Optimizer)

# Yields associated with different application rates (kg/ha)
pest_yields = Dict(0 => (2900, 3500, 5900), 1 => (3800, 4100, 6700), 2 => (4400, 4200, 7900))

# Production Costs $/ha-yr (from Table 2)
C_s = 350
C_w = 280
C_c = 390

# Selling Prices $/kg (from Table 2)
P_s = 0.36
P_w = 0.27
P_c = 0.22

@variable(crop_model, A_s0 >= 0) 
@variable(crop_model, A_s1 >= 0)
@variable(crop_model, A_s2 >= 0)

@variable(crop_model, A_w0 >= 0)
@variable(crop_model, A_w1 >= 0)
@variable(crop_model, A_w2 >= 0)

@variable(crop_model, A_c0 >= 0)
@variable(crop_model, A_c1 >= 0)
@variable(crop_model, A_c2 >= 0)

@constraint(crop_model, A_s0+A_s1+A_s2 + A_w0+A_w1+A_w2 + A_c0+A_c1+A_c2 <= 130)

# Objective
@objective(crop_model, Max,
    (pest_yields[0][1] * A_s0 * P_s + 
     pest_yields[1][1] * A_s1 * P_s + 
     pest_yields[2][1] * A_s2 * P_s + 
     pest_yields[0][2] * A_w0 * P_w + 
     pest_yields[1][2] * A_w1 * P_w + 
     pest_yields[2][2] * A_w2 * P_w + 
     pest_yields[0][3] * A_c0 * P_c + 
     pest_yields[1][3] * A_c1 * P_c + 
     pest_yields[2][3] * A_c2 * P_c) - 
    (C_s * (A_s0 + A_s1 + A_s2) + 
     C_w * (A_w0 + A_w1 + A_w2) + 
     C_c * (A_c0 + A_c1 + A_c2) + 
     70*((A_s1 + 2*A_s2) + (A_w1 + 2*A_w2) + (A_c1 + 2*A_c2)))
)

@constraint(crop_model, (0*A_s0 + 1*A_s1 + 2*A_s2)/130 <= 0.8)
@constraint(crop_model, (0*A_w0 + 1*A_w1 + 2*A_w2)/130 <= 0.7)
@constraint(crop_model, (0*A_c0 + 1*A_c1 + 2*A_c2)/130 <= 0.6)

@constraint(crop_model, pest_yields[0][1] * A_s0 + pest_yields[1][1] * A_s1 + pest_yields[2][1] * A_s2 <= 250000)
@constraint(crop_model, pest_yields[0][2] * A_w0 + pest_yields[1][2] * A_w1 + pest_yields[2][2] * A_w2 <= 250000)
@constraint(crop_model, pest_yields[0][3] * A_c0 + pest_yields[1][3] * A_c1 + pest_yields[2][3] * A_c2 <= 250000)

optimize!(crop_model)

println("Area of soybeans with 0 rate: ", value(A_s0))
println("Area of soybeans with 1 rate: ", value(A_s1))
println("Area of soybeans with 2 rate: ", value(A_s2))

println("Area of wheat with 0 rate: ", value(A_w0))
println("Area of wheat with 1 rate: ", value(A_w1))
println("Area of wheat with 2 rate: ", value(A_w2))

println("Area of corn with 0 rate: ", value(A_c0))
println("Area of corn with 1 rate: ", value(A_c1))
println("Area of corn with 2 rate: ", value(A_c2))

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-03, 8e+03]
  Cost   [7e+02, 1e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e-01, 2e+05]
Presolving model
7 rows, 9 cols, 24 nonzeros  0s
7 rows, 9 cols, 24 nonzeros  0s
Presolve : Reductions: rows 7(-0); columns 9(-0); elements 24(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -5.0012461107e+02 Ph1: 7(21.524); Du: 9(500.125) 0s
          4     1.3050427690e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  1.3050427690e+05
HiGHS run time      :          0.00
Area of soybeans with 0 rate: 0.0
Area of soybeans with 1 rate: 13.250000000000009
Area of soybeans with 2 rate: 45.37499999999999
Area of wheat with 0 rate: 0.0
Area of wheat with 1 rate: 39.72943037974684
Area of wheat with 2 rate: 0.0
Area o

### Problem 2 (30 points)

For this problem, we will use hourly load (demand) data from 2013 in New
York’s Zone C (which includes Ithaca). The load data is loaded and
plotted below in <a href="#fig-demand" class="quarto-xref">Figure 1</a>.

In [1]:
# load the data, pull Zone C, and reformat the DataFrame
NY_demand = DataFrame(CSV.File("data/2013_hourly_load_NY.csv"))
rename!(NY_demand, :"Time Stamp" => :Date)
demand = NY_demand[:, [:Date, :C]]
rename!(demand, :C => :Demand)
demand[:, :Hour] = 1:nrow(demand)

# plot demand
plot(demand.Hour, demand.Demand, xlabel="Hour of Year", ylabel="Demand (MWh)", label=:false)

Next, we load the generator data, shown in
<a href="#tbl-generators" class="quarto-xref">Table 3</a>. This data
includes fixed costs (\$/MW installed), variable costs (\$/MWh
generated), and CO<sub>2</sub> emissions intensity (tCO<sub>2</sub>/MWh
generated).

In [1]:
gens = DataFrame(CSV.File("data/generators.csv"))

Finally, we load the hourly solar and wind capacity factors, which are
plotted in <a href="#fig-cf" class="quarto-xref">Figure 2</a>. These
tell us the fraction of installed capacity which is expected to be
available in a given hour for generation (typically based on the average
meteorology).

In [1]:
# load capacify factors into a DataFrame
cap_factor = DataFrame(CSV.File("data/wind_solar_capacity_factors.csv"))

# plot January capacity factors
p1 = plot(cap_factor.Wind[1:(24*31)], label="Wind")
plot!(cap_factor.Solar[1:(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

p2 = plot(cap_factor.Wind[4344:4344+(24*31)], label="Wind")
plot!(cap_factor.Solar[4344:4344+(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

display(p1)
display(p2)

You have been asked to develop a generating capacity expansion plan for
the utility in Riley County, NY, which currently has no existing
electrical generation infrastructure. The utility can build any of the
following plant types: geothermal, coal, natural gas combined cycle gas
turbine (CCGT), natural gas combustion turbine (CT), solar, and wind.

While coal, CCGT, and CT plants can generate at their full installed
capacity, geothermal plants operate at maximum 85% capacity, and solar
and wind available capacities vary by the hour depend on the expected
meteorology. The utility will also penalize any non-served demand at a
rate of \$10,000/MWh.

**In this problem**:

-   Formulate a linear program for this capacity expansion problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations).
-   Implement your linear program in `JuMP.jl`. Find the optimal
    solution. How much should the utility build of each type of
    generating plant? What will the total cost be? How much energy will
    be non-served?
-   What fraction of annual generation does each plant type produce? How
    does this compare to the breakdown of built capacity that you found
    in Problem 1.5? Do these results make sense given the generator
    data?
-   Make a plot of the electricity price in each hour. Discuss any
    trends that you see.

> **Significant Digits**
>
> Use `round(x; digits=n)` to report values to the appropriate
> precision! If your number is on a different order of magnitude and you
> want to round to a certain number of significant digits, you can use
> `round(x; sigdigits=n)`.

> **Getting Variable Output Values**
>
> `value.(x)` will report the values of a `JuMP` variable `x`, but it
> will return a special container which holds other information about
> `x` that is useful for `JuMP`. This means that you can’t use this
> output directly for further calculations. To just extract the values,
> use `value.(x).data`.

> **Suppressing Model Command Output**
>
> The output of specifying model components (variable or constraints)
> can be quite large for this problem because of the number of time
> periods. If you end a cell with an `@variable` or `@constraint`
> command, I *highly* recommend suppressing output by adding a
> semi-colon after the last command, or you might find that your
> notebook crashes.

## References

List any external references consulted, including classmates.

- Problem 1: How to create a dictionary in Julia (for Table 1 values): https://discourse.julialang.org/t/create-a-dictionary-from-arrays-of-keys-and-values/13908 